In [1]:
import sqlite3 as lite
import os as os
from glob import glob
import pandas as pd
con=lite.connect('1RVU22BSC018_chandanab')

In [2]:
con.execute("create Table IF NOT EXISTS PRODUCT_CBK (PRODUCTID NOT NULL, PRODUCTNAME TEXT NOT NULL, SupplierID TEXT NOT NULL, CategoryID INTEGER NOT NULL, Unit TEXT NOT NULL, Price INTEGER NOT NULL)")

In [3]:
PRODUCT_CBK = [
    (1, 'Chais', 1, 1, '10 boxes x 20 bags', 18.0),
    (2, 'Chang', 1, 1, '24 - 12 oz bottles', 19.0),
    (3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0),
    (4, "Chef Anton's Cajun Seasoning", 2, 2, '48 - 6 oz jars', 22.0),
    (5, "Chef Anton's Gumbo Mix", 2, 2, '36 boxes', 21.35),
    (6, "Grandma's Boysenberry Spread", 3, 2, '12 - 8 oz jars', 25.0),
    (7, "Uncle Bob's Organic Dried Pears", 3, 7, '12 - 1 lb pkgs.', 30.0),
    (8, "Northwoods Cranberry Sauce", 3, 2, '12 - 12 oz jars', 40.0),
    (9, "Mishi Kobe Niku", 4, 6, '18 - 500 g pkgs.', 97.0)
]
query = """
  INSERT INTO PRODUCT_CBK
  VALUES(?,?,?,?,?,?)
"""

con.executemany(query, PRODUCT_CBK)
pd.read_sql("Select * from PRODUCT_CBK",con)

,PRODUCTID,PRODUCTNAME,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.00
1,2,Chang,1,1,24 - 12 oz bottles,19.00
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35
5,6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25.00
6,7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.00
7,8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40.00
8,9,Mishi Kobe Niku,4,6,18 - 500 g pkgs.,97.00


In [4]:
con.execute("CREATE TABLE IF NOT EXISTS OrderDetails_CBK(OrderDetailID INTEGER PRIMARY KEY, OrderID INTEGER NOT NULL,ProductID INTEGER NOT NULL,Quantity INTEGER NOT NULL)")

In [5]:
OrderDetails_CBK = [
    (1, 10248, 2, 12),
    (2, 10248, 4, 10),
    (3, 10248, 5, 5),
    (4, 10249, 2, 9),
    (5, 10249, 2, 40),
    (6, 10250, 2, 10),
    (7, 10250, 6, 35),
    (8, 10250, 7, 15),
    (9, 10251, 8, 6),
    (10, 10251, 20, 15)

]
query = """
  INSERT INTO OrderDetails_CBK
  VALUES(?,?,?,?)
"""

con.executemany(query, OrderDetails_CBK)
pd.read_sql("Select * from OrderDetails_CBK",con)

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,2,12
1,2,10248,4,10
2,3,10248,5,5
3,4,10249,2,9
4,5,10249,2,40
5,6,10250,2,10
6,7,10250,6,35
7,8,10250,7,15
8,9,10251,8,6
9,10,10251,20,15


In [ ]:
query1 = """
SELECT ProductName
FROM Products_chandanab
WHERE ProductID IN (
    SELECT ProductID
    FROM OrderDetails_chandanab
    GROUP BY ProductID
    HAVING SUM(Quantity) > (SELECT MIN(Quantity) FROM OrderDetails_chandanab)
);

"""
pd.read_sql_query(query1, con)

In [ ]:
query2 = """
SELECT ProductName
FROM Products_CBK
WHERE EXISTS (
    SELECT 1
    FROM OrderDetails_CBK
    WHERE Products_Mithun.ProductID = OrderDetails_CBK.ProductID
    AND Quantity < (
        SELECT AVG(Quantity)
        FROM OrderDetails_CBK   )
);

"""
pd.read_sql_query(query2, con)

In [ ]:
query3 = """
SELECT o.OrderID, GROUP_CONCAT(p.ProductName) AS ProductNames
FROM OrderDetails_CBK o
JOIN OrderDetails_CBK od ON o.OrderID = od.OrderID
JOIN Products_CBK p ON od.ProductID = p.ProductID
GROUP BY o.OrderID;

"""
pd.read_sql_query(query3, con)

In [ ]:
query4 = """ SELECT *
FROM OrderDetails_CBK
WHERE OrderID IN (
    SELECT OrderID
    FROM OrderDetails_CBK
    GROUP BY OrderID
    HAVING COUNT(*) > 1
);
"""
pd.read_sql_query(query4,con)